# Simultaneous Imitation (Scrap)

## Abstract

## Subject heatmap and Grand Average

We calculate the grand average data heatmap and with only fixation.


In [ ]:
# TODO: make a function that can plot intervals of time or frames to check center bias
print("Hola!")

## Cut acc signal according to trials frames (checkout if matlab was done right)


In [ ]:
# TODO: Make a function that cuts signal according to frames in trial, remember that acc signal is in time, 
# we need TTL = when acc was ON in frame
print("!")